In [1]:
from redditscore.tokenizer import CrazyTokenizer
tokenizer = CrazyTokenizer(hashtags='split', twitter_handles='realname')
tokenizer.tokenize('@realdonaldtrump please #MakeAmericaGreatAgain')
#['donald', 'j.', 'trump', 'please', 'make', 'america', 'great', 'again']

['donald', 'j.', 'trump', 'please', 'make', 'america', 'great', 'again']

In [2]:
import pandas as pd
import sys
import json
import os
import extraction
import requests
from redditscore.tokenizer import CrazyTokenizer
import nltk
import re
import requests
from tqdm.notebook import tqdm
import numpy as np
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import warnings
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('punkt') # if necessary...
pd.set_option('display.max_colwidth', 255)
warnings.filterwarnings("ignore")
nltk.download('stopwords')

/Users/Sumon/opt/anaconda3/envs/nlp/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


[nltk_data] Downloading package punkt to /Users/Sumon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/Sumon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

In [38]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
train['text'] = train['text'].apply(lambda x:clean_text(x))
test['text'] = test['text'].apply(lambda x:clean_text(x))

In [39]:
train['text_len'] = train['text'].astype(str).apply(len)
train['text_word_count'] = train['text'].apply(lambda x: len(str(x).split()))

In [40]:
train

,text,immigration,text_len,text_word_count
0,mr chairman such a trial is a farce reference was made here to bringing people from farflung sections of this country every time a federal judge is investigated it means a denial of justice to a lot of american people who are entitled to be relieved o...,0.0,6888,1238
1,mr president i have for some time now been keenly interested in the competitiveness of us firms in global markets as we look to the century there can be no doubt that the future growth of the us economy depends to a very large extent on the ability o...,0.0,5383,868
2,madam president i thank the senator from rhode island for yielding i just want to talk a little bit about data i believe in data i try to find good information about issues and use that information to inform my work we need good data but can we trust ...,1.0,6153,1047
3,all aliens who have violated our laws since,1.0,44,8
4,my friend from new york is an able profound lawyer but not so good a parliamentarian rumors have reached me that at a meeting of the committee on immigration and naturalization yesterday the chairman of the committee the gentleman from new york attem...,1.0,1140,207
...,...,...,...,...
1795,while the distinguished senior senator from arizona is still on the floor i would like to note that during the process of putting this bill together in the committee and having the votes we had a number of quiet meetings bipartisan meetings in the pre...,0.0,2097,387
1796,mr speaker our country our declaration of independence and our constitution are all based on a promise the promise in the declaration of independence is that taxation without representation was and is wrong the promise in our constitution is that all ...,0.0,2794,484
1797,mr chairman i yield myself such time as i may consume mr chairman i am pleased that at long last the house is considering the bill to authorize appropriations for the department of justice for fiscal year i wish to compliment chairman rodino who pres...,1.0,3023,516
1798,mr speaker i yield myself such time as i may consume mr speaker under the immigration and nationality act a program exists which authorizes religious denominations throughout the united states to sponsor nonminister workers in religious vocations and ...,1.0,2628,402


In [41]:
# Visualisation libraries
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import chart_studio.plotly as py
import plotly.figure_factory as ff
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

train['text_word_count'].iplot(
    kind='bar',
    bins=50,
    xTitle='text length',
    linecolor='black',
    color='red',
    yTitle='count',
    title='Positive Text word count')

In [42]:
train.describe()

,immigration,text_len,text_word_count
count,1800.000000,1.800000e+03,1800.000000
mean,0.562778,6.094829e+03,1062.717222
std,0.496181,6.144646e+04,10998.086565
min,0.000000,1.700000e+01,3.000000
25%,0.000000,1.005250e+03,175.000000
50%,1.000000,2.594500e+03,445.500000
75%,1.000000,5.829500e+03,984.250000
max,1.000000,2.586889e+06,463110.000000
